In [8]:
# import system libs
import os
import time
import shutil
import pathlib
import itertools
from PIL import Image

# import data handling tools
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
# import Deep learning Libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras import regularizers

train_dir = 'C:/Users/navan/Downloads/archive/Alzheimer_s Dataset/train'
filepaths = []
labels = []


loaded_model = tf.keras.models.load_model('C:/Users/navan/Downloads/model.h5', compile=False)
loaded_model.compile(Adamax(learning_rate= 0.001), loss= 'categorical_crossentropy', metrics= ['accuracy'])


folds = os.listdir(train_dir)
for fold in folds:
    foldpath = os.path.join(train_dir, fold)
    filelist = os.listdir(foldpath)
    for file in filelist:
        fpath = os.path.join(foldpath, file)
        filepaths.append(fpath)
        labels.append(fold)

# Concatenate data paths with labels into one dataframe
Fseries = pd.Series(filepaths, name= 'filepaths')
Lseries = pd.Series(labels, name='labels')
train_df = pd.concat([Fseries, Lseries], axis= 1)

# Generate  test data paths with labels
test_dir = 'C:/Users/navan/Downloads/archive/Alzheimer_s Dataset/test'
filepaths = []
labels = []

folds = os.listdir(test_dir)
for fold in folds:
    foldpath = os.path.join(test_dir, fold)
    filelist = os.listdir(foldpath)
    for file in filelist:
        fpath = os.path.join(foldpath, file)
        filepaths.append(fpath)
        labels.append(fold)

# Concatenate data paths with labels into one dataframe
Fseries = pd.Series(filepaths, name= 'filepaths')
Lseries = pd.Series(labels, name='labels')
test_df = pd.concat([Fseries, Lseries], axis= 1)

# valid and test dataframe
valid_df, test_df = train_test_split(test_df,  train_size= 0.6, shuffle= True, random_state= 123)


# cropped image size
batch_size = 16
img_size = (224, 224)
channels = 3
img_shape = (img_size[0], img_size[1], channels)

tr_gen = ImageDataGenerator()
ts_gen = ImageDataGenerator()
train_gen = tr_gen.flow_from_dataframe( train_df, x_col= 'filepaths', y_col= 'labels', target_size= img_size, class_mode= 'categorical',
                                    color_mode= 'rgb', shuffle= True, batch_size= batch_size)

valid_gen = ts_gen.flow_from_dataframe( valid_df, x_col= 'filepaths', y_col= 'labels', target_size= img_size, class_mode= 'categorical',
                                    color_mode= 'rgb', shuffle= True, batch_size= batch_size)

test_gen = ts_gen.flow_from_dataframe( test_df, x_col= 'filepaths', y_col= 'labels', target_size= img_size, class_mode= 'categorical',
                                    color_mode= 'rgb', shuffle= False, batch_size= batch_size)


g_dict = train_gen.class_indices      # defines dictionary {'class': index}
classes = list(g_dict.keys()) 

Found 5121 validated image filenames belonging to 4 classes.
Found 767 validated image filenames belonging to 4 classes.
Found 512 validated image filenames belonging to 4 classes.


In [9]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import numpy as np
import tensorflow as tf

class AlzheimerPredictionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Alzheimer's Prediction App")

        # Create GUI elements
        self.label = tk.Label(root, text="Select an image:")
        self.label.pack(pady=10)

        self.image_label = tk.Label(root)
        self.image_label.pack(pady=10)

        self.browse_button = tk.Button(root, text="Browse", command=self.browse_image)
        self.browse_button.pack(pady=10)

        self.predict_button = tk.Button(root, text="Predict", command=self.predict_image)
        self.predict_button.pack(pady=10)

        self.result_label = tk.Label(root, text="")
        self.result_label.pack(pady=10)

    def browse_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.png;*.jpg;*.jpeg")])
        if file_path:
            self.display_image(file_path)

    def display_image(self, file_path):
        image = Image.open(file_path)
        image = image.resize((224, 224))
        photo = ImageTk.PhotoImage(image)
        self.image_label.config(image=photo)
        self.image_label.image = photo
        self.selected_image = file_path

    def preprocess_image(self, image_path):
        img = Image.open(image_path)
        img = img.resize((224, 224))
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0)
        return img_array

    def predict_image(self):
        if hasattr(self, 'selected_image'):
            image_path = self.selected_image
            preprocessed_image = self.preprocess_image(image_path)
            predictions = loaded_model.predict(preprocessed_image)
            class_indices = np.argmax(predictions, axis=1)
            # Assume that class indices are in sequential order starting from 0
            class_labels = classes
            score = tf.nn.softmax(predictions[0])
            predicted_class = class_labels[tf.argmax(score)]
            self.result_label.config(text=f"Prediction: {predicted_class}")
        else:
            self.result_label.config(text="Please select an image first.")

# Create and run the app in a Jupyter Notebook cell
root = tk.Tk()
app = AlzheimerPredictionApp(root)
root.mainloop()


1/1 [==============================] - 1s 843ms/step


In [14]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import numpy as np
import tensorflow as tf

# Load the pre-trained model
loaded_model = tf.keras.models.load_model('model.h5', compile=False)

class_info = {
    "Mild-Demented": "\n Mild cognitive decline. Early-stage Alzheimer's.",
    "Moderate-Demented": "\n Moderate cognitive decline. Mid-stage Alzheimer's.",
    "Non-Demented": "\n No cognitive decline. Healthy.",
    "Very-Mild-Demented": "\n Very mild cognitive decline. Early-stage Alzheimer's."
}

class_labels = list(class_info.keys())

class AlzheimerPredictionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Alzheimer's Prediction App")
        self.root.geometry(f"{root.winfo_screenwidth()}x{root.winfo_screenheight()}")
        
        # Create GUI elements
        self.label = tk.Label(root, text="Select an image:")
        self.label.pack(pady=10)

        self.image_label = tk.Label(root)
        self.image_label.pack(pady=10)

        self.browse_button = tk.Button(root, text="Browse", command=self.browse_image)
        self.browse_button.pack(pady=10)

        self.predict_button = tk.Button(root, text="Predict", command=self.predict_image)
        self.predict_button.pack(pady=10)

        self.result_label = tk.Label(root, text="", font=("Helvetica", 20))
        self.result_label.pack(pady=10)

    def browse_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.png;*.jpg;*.jpeg")])
        if file_path:
            self.display_image(file_path)

    def display_image(self, file_path):
        image = Image.open(file_path)
        image = image.resize((224, 224))
        photo = ImageTk.PhotoImage(image)
        self.image_label.config(image=photo)
        self.image_label.image = photo
        self.selected_image = file_path

    def preprocess_image(self, image_path):
        img = Image.open(image_path)
        img = img.resize((224, 224))
        
        # Remove alpha channel if it exists
        if img.mode == 'RGBA':
            img = img.convert('RGB')
            
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0)
        return img_array

    def predict_image(self):
        if hasattr(self, 'selected_image'):
            image_path = self.selected_image
            preprocessed_image = self.preprocess_image(image_path)
            predictions = loaded_model.predict(preprocessed_image)
            class_indices = np.argmax(predictions, axis=1)
            
            # Assume that class indices are in sequential order starting from 0
            predicted_class = class_labels[class_indices[0]]
            prediction_info = class_info[predicted_class]

            self.result_label.config(text=f"Prediction: {predicted_class}\n{prediction_info}")
        else:
            self.result_label.config(text="Please select an image first.")

# Create and run the app
root = tk.Tk()
app = AlzheimerPredictionApp(root)
root.mainloop()

1/1 [==============================] - 1s 680ms/step
